In [ ]:
import numpy as np
import pandas as pd
import glob
import sqlite3
import re
import os
import resource

In [ ]:
def read_counts(csv_path):
    base_name = re.sub("_PPI_barcodes_fitness_counts.csv","",os.path.basename(csv_path))
    print("Beginning to read and process counts of ",base_name)
    this_datar = pd.read_csv(csv_path)
    print("Read CSV for ",base_name)
    this_datar.rename(columns={
            "Fitness_estimaion_error(d)":"Error",
            "Fitness_estimaion_error":"Error"
        },inplace=True)
    this_datar.rename(columns=lambda x: re.sub("Counts_G","",x),inplace=True)
    this_datar = this_datar.melt(id_vars=["PPI"], value_vars=this_datar.filter(regex="\d+").columns)
    this_datar.rename(columns={"value":"counts","variable":"generation"},inplace=True)
    print("Melted for ",base_name)
    this_datar["Experiment"] = base_name
    print(this_datar)
    return this_datar
#,nrows=100)\
#for debugging, this makes it run quite a bit faster...

In [ ]:
print("Opening a SQLite database at `counts.sqlite` ")
db_counts = sqlite3.connect("counts.sqlite")

for this_csv in glob.glob('*_PPI_*.csv'):
    with db_counts:
        print('Memory usage: %s (kb)' % resource.getrusage(resource.RUSAGE_SELF).ru_maxrss)
        read_counts(this_csv)\
            .to_sql("counts",db_counts,if_exists="append",chunksize=100,index=False)

print("Database on disk reports these heads for counts:")
#print(pd.read_sql_query("SELECT * FROM counts LIMIT 5",db_counts))